In [ ]:
from huggingface_hub import login
from esm.models.esm3 import ESM3
from esm.sdk.api import ESM3InferenceClient, ESMProtein, GenerationConfig

# # Will instruct you how to get an API key from huggingface hub, make one with "Read" permission.
# login()

# This will download the model weights and instantiate the model on your machine.
model: ESM3InferenceClient = ESM3.from_pretrained("esm3_sm_open_v1").to("cuda") # or "cpu"

In [56]:
# Generate a completion for a partial Carbonic Anhydrase (2vvb)
sequence = "MGSIQQDDVHNQIDHCNQSDDLPAARLNCNDVELFEVAGLACDETSSPTGMRDEMVLLSWLIALLRTREGGQIRYEWAYRYPEEEPVPRCLAMNEVVAGLQSSVKETAAAVSRHISADVSSPPAPASLLLSTSSLSQTSDEAKDEGLLHLEIAFENGLCKIRPTWHSENMLPFTVTRYARTLIDTVRLCISNCDAAIQDCLRPTAYDLDEIWRWNHNLPPTYNFCMHEIISDQAQKFPDKEAIASWDGSLTYRQIDQYSSFVARSLIGMGVGLHDVLPVCFEKSRWTIVAVLAVMKAGATFVLMDPTLPLARLQNMAQQVGAKMMVSSRGQYNLATEIIPNANVLVVEENTFSSLSAEQNGEPLPTVPSSALMYMIFTSGSTGTPKGVKISHETYTSSAIPRANAVGYTEDSRVLDFASYAFDVSIDSMLLTLGNGGCLCIPSDEDRLNDINGVIRRMKVNYAGLTPSVARILDADVISSLSGLGLGGEAVSARDVNLWGQDTRIIIGYGPCECTIGCTVNSSAATGRDYISIGPGNGAVIWIVDPNDHESLVPLGAVGELLVEGPIVGQGYLNDPEKTAAAFIEDPSWLVAGHEGYPGRRGRLYKTGDLGRYDPDGSGGIVFVGRKDTQVKLRGQRVELGEIESQLRARLPSETTVIAEVIVPQGSGGQPTLVAFVAAQTTKGHDHTGLEAAELPDELRRALSEADAELAKVLPRYMVPTAYIPVNHIPTLISGKTDRKRLRQFGATVDLRQLDQDATNTAARELSDLERRLRQAWSQTLKLQACSIRLQDNFFALGGDSLTAMKLVSVCRSQGLDLSVTSMFSNPTLSAMASVVRICDVDVQRTVPAFSMITSDMNSACVEAAEPCGVGPADIEDIYPCTPTQESLFTFSLKSVKPYVAQRVLCIPSHIDLNAWRKAWEDVVAALPILRTRVAQLQEPGLQQVVLKNSISWTQASDLAEYLENDRTQKMNLGESLARYAIVEDSADGKRYMVWTIHHVLYDGWSEPIILKQVSDALQGQPVEVKAQMRDFVRFVRDSDDAAVQEFWRRELKGAVGPQFPRLPSRDFMPTPDALVERQVSLDTSSGSPFTMATLIRGAWALVASQYTGSDDIVFGETLTGRDIPLPGVESIVGPLIATVPIRVRILRGSTVESYLQAVQQSVLARTPYQHLGMQNIRKVSQDAQHACETGTGLVIQPEPEYVGSELGVERGDVVLEALHFNPYPLMLACGIRKGGFRVCASFDSSLIETRQMERMLAQLETACWQLSQGLSRKVDEISCLPEAELNQIWQWNRSPPLSLDETTSRLRANASTKPGSSYPPAVVPWVCSPRNSSLLSPIGCVGELWLEGALLSGDTVDSPAWLVAGSSTCAGRTGKVQATGDMVQLREDGSLVFVGRKENVVPVQGHAVDITEIERHLAEHLPPTIRAAATVVRSSSDQELVMFIEQPAAEEACIELLSEKREIVCDAPDKAFQTTICATIPGSLAAVLKKLDKYMRDSLPSYMAPSAYIVVEKLPNTMDDIDHNLLNQIASQVTPQILNELRDGLSNAWTKATAPNHLSASESILRSAWAKVLRVDPEQIDVDDNFFRRGGDSVLAMKLVSSLRAQGYSLSVADIFRHMRLSDAARVMKVDERSTEKINSYQPFSMLRLPDVQQFLANIVRPQLGDQHWPIRDVLPVTDSQDMDIRATIQPPRTSIQYTMLYFDNSVDRERLFRSCSDLVKTHEILRTVFISHESSFLQVVLNELEIPVRAHKTDKQLDQYVASLFREDIESNFQLGCPFLRLFYVEGNNGESCLVIGLSHAQYDGVSLPRLLQDLDALYTGTQLATFSPFSLYMAQTSEEAIQNKAAAYWRNLLSSSSLSTLDGPSSDPTDKAIFHTRPVNIHPLKEITTANLLTAAWAMVLARRLQTPDVTFGSVTSGRTLDIPNAENFMGPCYQLTPVRVPFHPDWTASDLLNFVQTQSAESAAHDFLGFEKIAKLAGWASGRQGFDSIVHHQDWEDFDMMPFGGGSCRVDIANPHGDAAYPVKAVSFVKEGEIHVGVVCSERDVMFVDEVLGELAAAVVELAGQSTEVLLDSKLFSGQ"
length = 360
divisor = 10

new_sequence = ""
for i, aa in enumerate(sequence):
    if i % divisor == 0:
        new_sequence += '_'
    else:
        new_sequence += aa
prompt = new_sequence[:length]

In [61]:
# Generate the sequence.
protein = ESMProtein(sequence=prompt)
protein = model.generate(protein, GenerationConfig(track="sequence", num_steps=10, temperature=1))

# protein = model.generate(protein, GenerationConfig(track="structure", num_steps=8))
# protein.sequence = None
# protein = model.generate(protein, GenerationConfig(track="sequence", num_steps=8))

output = protein.sequence

100%|██████████| 10/10 [00:14<00:00,  1.45s/it]


In [62]:
mistakes = 0
for i, (aa1, aa2) in enumerate(zip(output, sequence)):
    if aa1 != aa2:
        mistakes += 1

print(f"Number of mistakes: {mistakes}")
print(f"percentage of correct: {(1 - mistakes / (length // divisor)) * 100:.2f}%")

Number of mistakes: 32
percentage of correct: 11.11%
